In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [233]:
# data
def linerMock(W, b, number):
    X = torch.normal(0, 1, (number, len(W)))
    y = torch.matmul(X, W) + b
    y += torch.normal(0, 0.5, y.shape)
    return X, torch.reshape(y, (-1, 1))
    # return X, y

feature, label = linerMock(torch.tensor([-2.0, 4.0]), 1.0, 1000)

In [241]:
torch.tensor([-2.0, 4.0]).shape

torch.Size([2])

In [ ]:
# generate data
# y = 3x + 2

def genData(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.1, y.shape)
    return X, torch.reshape(y, (-1,1))

feature, label = genData(torch.tensor([-2.0, 4.0]), 2.0, 1000)

d2l.plt.scatter(feature[:, 1].detach(), label.detach())

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
feature, label = d2l.synthetic_data(true_w, true_b, 1000)
d2l.plt.scatter(feature[:, 1].detach(), label.detach())

In [7]:
#hyper parameter
lr = 0.03
batchSize = 32
epoch = 5

In [218]:
myDataset = data.TensorDataset(*(feature, label))
dataItr = data.DataLoader(myDataset, batchSize, shuffle=True)

In [108]:
print(next(iter(dataItr)))

[tensor([[-0.7776, -0.6059],
        [ 0.0396,  1.3283],
        [-1.0419, -1.3931],
        [-0.1975, -1.0258],
        [ 0.7046,  0.0366],
        [ 1.4410, -0.9633],
        [-0.7207, -0.9129],
        [-1.1566, -0.6565],
        [-0.0031,  0.2703],
        [-0.6154, -2.1001],
        [-0.8926, -0.7285],
        [ 0.6367, -0.8788],
        [-0.1056,  1.1227],
        [ 0.8463,  0.8790],
        [ 0.7417, -1.2538],
        [ 0.4993, -0.1989],
        [-0.0152,  0.4799],
        [-1.2056,  0.7613],
        [ 0.4834,  0.6815],
        [-0.8112, -0.2447],
        [-0.8295, -0.2632],
        [-1.0830,  1.0580],
        [-0.5661,  0.1275],
        [ 0.0824, -0.6888],
        [-2.0770,  0.0330],
        [-1.8357, -0.7490],
        [ 1.2992, -2.2128],
        [-0.8101,  0.9993],
        [ 1.6277,  0.0930],
        [-1.0369, -0.6421],
        [ 0.0851,  0.6547],
        [ 0.5268, -2.0138]]), tensor([[ 4.7001],
        [-0.2107],
        [ 6.8593],
        [ 7.2879],
        [ 5.4932],
      

In [64]:
from torch import nn

class MyNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten : nn.Flatten = nn.Flatten()
        self.linear : nn.Sequential = nn.Sequential(nn.Linear(2,1))

    def forward(self, x):
        # x = self.flatten(x)
        return self.linear(x)

model = MyNet().to("cpu")

In [220]:
print(model.linear[0].weight.data.normal_(0, 0.1))
print(model.linear[0].bias.data.fill_(0))

tensor([[ 0.0631, -0.0308]])
tensor([0.])


In [66]:
lossFn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [128]:
def trainLoop(dataloader: data.DataLoader, 
              model: nn.Module, 
              lossFn: nn.MSELoss, optimizer: torch.optim.SGD):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = lossFn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batchSize + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            l = lossFn(model(feature), label)
            print(f'epoch {epoch + 1}, loss {l:f}')

In [221]:
for t in range(epoch):
    print(f"Epoch {t+1}\n")
    trainLoop(dataItr, model, lossFn, optimizer)

Epoch 1

loss: 17.439655 [   32/ 1000]
epoch 3, loss 18.553246
Epoch 2

loss: 0.726910 [   32/ 1000]
epoch 3, loss 0.766165


In [219]:
net = model.linear
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in dataItr:
        l = lossFn(net(X) ,y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    l = lossFn(net(feature), label)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.718117
epoch 2, loss 0.295574
epoch 3, loss 0.285686
